In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
hyper_epoch=50

In [3]:
thresholds={0: 0.0,\
            1:-0.5,\
            2:-0.9,\
            3:-1.7,\
            4:-1.7,\
            5:-0.8,\
            6:-1.7,\
            7:-2.0}

In [ ]:
for _ in range(hyper_epoch):
    # model parameters
    reuse = np.random.choice([True,False],p=[0.2,0.8])
    block = np.random.choice([NNConv_block,schnet_block],p=[0.6,0.4])
    head = np.random.choice([cat3Head_type,swapHead_type,cat3HeadPool_type,set2setHead_type,SimpleHead_type],\
                            p=[0.1875,0.1875,0.1875,0.1875,0.25])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA.pickle',\
                             '../Data/{}_data_atomInfo.pickle',\
                             '../Data/{}_data_ACSF_SOAP_atomInfo.pickle'])
    aggr = np.random.choice(['mean','add','max'])
    
    batch_size = 32
    dim = int(np.random.choice([128,256,512]))
    epochs = 50
    clip = 2
    interleave = np.random.choice([True,False])
    if interleave:
        layer1 = int(np.random.choice([2,3]))
        layer2 = layer1
    else:
        layer1 = int(np.random.choice([2,3]))
        layer2 = int(np.random.choice([2,3]))
    factor = int(np.random.choice([2,3]))
    lr = 1e-4
    epochs_type = epochs

    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [reuse,block,head,data,batch_size,dim,\
                                              layer1,layer2,factor,aggr,interleave]])))

    for i in range(8):
        # load type data
        data_type = data.split('.pickle')[0]+ '_type_'+str(i)+'.pickle'
        train_dl,val_dl = get_data(data_type,batch_size)

        # init model
        model = GNN(reuse,block,head,dim,layer1,layer2,factor,\
                    **data_dict[data],aggr=aggr,interleave=interleave).to('cuda:0')
        paras = trainable_parameter(model)
        opt = Adam(paras,lr=lr)
        scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

        print('\nTraining type: {}'.format(i))
        model,train_loss_perType,val_loss_perType = train_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,\
                                                                    scheduler=scheduler,threshold=thresholds[i])
        
        if model is not None:
            save_results2(train_loss_perType,val_loss_perType,reuse,block,\
                          head,data,dim,layer1,layer2,factor,aggr,interleave,'type_'+str(i)+'_start')

            #save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,aggr,'type_'+str(i)+'_start')